In [68]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

/Users/pipipu/miniconda3/envs/DLML/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [155]:
import torch
import torch.nn as nn

class Type_Embedding(torch.nn.Module):
    def __init__(self, seq_len, hidden_dim, output_dim):
        super(Type_Embedding, self).__init__()
        self.seq_len = seq_len
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.MLP1 = nn.Linear(self.seq_len, self.hidden_dim)
        self.ac = nn.GELU()
        self.MLP2 = nn.Linear(self.hidden_dim, self.output_dim)
    
    def forward(self, x):
        x = self.MLP1(x)
        x = self.ac(x)
        x = self.MLP2(x)
        return x


In [225]:
import torch
import torch.nn as nn

class Separate_Embedding(torch.nn.Module):
    def __init__(self, seq_len, hidden_dim, output_dim):
        super(Separate_Embedding, self).__init__()
        self.seq_len = seq_len
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        self.linear_layers = nn.ModuleList([
            Type_Embedding(self.seq_len, self.hidden_dim,self.output_dim) for _ in range(10)
        ])
       
    def forward(self, x):
        tensor_list = torch.split(x, 1, dim=0)
        processed_tensors = []

        for i, tensor in enumerate(tensor_list):
            linear_layer = self.linear_layers[i]
            processed_tensor = linear_layer(tensor)
            processed_tensors.append(processed_tensor)
        
        combined_tensor = torch.cat(processed_tensors, dim=0)
        return combined_tensor


In [215]:
# Create an instance of Separate_Embedding
separate_embedding = Separate_Embedding(seq_len=8, hidden_dim=16, output_dim=4)

# Example input tensor
input_tensor = torch.randn(8, 133, 8)

# Apply the separate embedding
output_tensor = separate_embedding(input_tensor)

# Print the output tensor
print(output_tensor.shape)


torch.Size([1, 133, 8])
torch.Size([8, 133, 4])


In [158]:
import torch
import torch.nn as nn

class Adj_Embedding(torch.nn.Module):
    def __init__(self, adj_matrix, hidden_dim_1, hidden_dim_2, output_dim, k):
        super(Adj_Embedding, self).__init__()
        self.adj_matrix = adj_matrix
        self.input_dim = self.adj_matrix.shape[0]
        self.hidden_dim_1 = hidden_dim_1
        self.hidden_dim_2 = hidden_dim_2
        self.output_dim = output_dim
        self.k = k

        self.MLP1_d1 = nn.Linear(self.input_dim, self.hidden_dim_1)
        self.ac1_d1 = nn.GELU()
        self.MLP2_d1 = nn.Linear(self.hidden_dim_1, self.output_dim)

        self.MLP1_d2 = nn.Linear(self.input_dim, self.hidden_dim_2)
        self.ac1_d2 = nn.GELU()
        self.MLP2_d2 = nn.Linear(self.hidden_dim_2, self.output_dim)
    
    def forward(self, x):
        x = self.MLP1_d1(x)
        x = self.ac1_d1(x)
        x = x.reshape(x.shape[1], x.shape[0])
        x = self.MLP1_d2(x)
        x = self.ac1_d2(x)
        x = self.MLP2_d2(x)
        x = x.reshape(x.shape[1], x.shape[0])
        x = self.MLP2_d1(x)
        
        softmax_output = torch.nn.functional.softmax(x, dim=0)
        topk_values, topk_indices = torch.topk(softmax_output, self.k, dim=0)
        masked_output = torch.where(softmax_output < topk_values[-1], torch.tensor(0.0), softmax_output)

        return masked_output


In [159]:
# Create an instance of Separate_Embedding
adj = torch.randn(10,10)
hidden_dim_1 = 32
hidden_dim_2 = 32 
output_dim = 10
k = 3
separate_embedding = Adj_Embedding(adj,hidden_dim_1,hidden_dim_2,output_dim,k)

# Example input tensor

# Apply the separate embedding
output_tensor = separate_embedding(adj)

# Print the output tensor
print(output_tensor.shape)


torch.Size([10, 10])


In [160]:
adj = np.load('/Users/pipipu/Desktop/HAGEN/HAGEN-code/crime-data/sensor_graph/adj_mx_la.pkl',allow_pickle=True)

In [217]:
class Linear(torch.nn.Module):
    def __init__(self, seq_len, hidden_dim, output_dim):
        super(Linear, self).__init__()
        self.seq_len = seq_len
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.MLP1 = nn.Linear(self.seq_len, self.hidden_dim)
        self.ac = nn.GELU()
        self.MLP2 = nn.Linear(self.hidden_dim, self.output_dim)
    
    def forward(self, x):
        x = self.MLP1(x)
        x = self.ac(x)
        x = self.MLP2(x)
        return x


In [228]:
class Framework(torch.nn.Module):
    def __init__(self, seq_len, hidden_dim_type, output_dim_type, adj_matrix, hidden_dim_1, hidden_dim_2, output_dim_adj, k, input_dim_main, hidden_dim_main,output_dim_main):
        super(Framework, self).__init__()
        self.seq_len = seq_len
        self.hidden_dim_type = hidden_dim_type
        self.output_dim_type = output_dim_type

        self.embedding = Separate_Embedding(self.seq_len, self.hidden_dim_type, self.output_dim_type)

        self.adj_matrix = adj_matrix
        self.matrix_dim = adj_matrix.shape[0]
        self.hidden_dim_1_adj = hidden_dim_1
        self.hidden_dim_2_adj = hidden_dim_2
        self.output_dim_adj = output_dim_adj
        self.k = k

        self.adj_embedding = Adj_Embedding(self.adj_matrix, self.hidden_dim_1_adj, self.hidden_dim_2_adj, self.output_dim_adj, self.k)

        self.input_dim_main = input_dim_main
        self.hidden_dim_main = hidden_dim_main
        self.output_dim_main = output_dim_main
        self.main = Linear(self.input_dim_main,self.hidden_dim_main,self.output_dim_main)

    def forward(self, x):
        
        x = self.embedding(x)
        
        x = x.permute(0,3,2,1)
        self.adj = self.adj_embedding(self.adj_matrix)
        
        x = self.main(x)
        x = x.permute(0,3,2,1)
        return x


In [280]:
X_data = np.load("/Users/pipipu/Desktop/HAGEN/HAGEN-code/crime-data/CRIME-LA/8/train.npz")

In [281]:
X = torch.from_numpy(X_data['x'])
y = torch.from_numpy(X_data['y'])

In [199]:
import torch
import torch.nn as nn
import torch.optim as optim

def train(model, train_loader, num_epochs, learning_rate):
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(num_epochs):
        running_loss = 0.0

        # Iterate over the training dataset
        for inputs, labels in train_loader:
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update the running loss
            running_loss += loss.item()

        # Print the average loss for the epoch
        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")

    print("Training finished!")



In [221]:
seq_len = 8
hidden_dim_type = 32
output_dim_type = 32
adj_matrix =  np.load('/Users/pipipu/Desktop/HAGEN/HAGEN-code/crime-data/sensor_graph/adj_mx_la.pkl',allow_pickle = True)[2]
hidden_dim_1 = 32
hidden_dim_2 = 32
output_dim_adj = 113
k = 3
input_dim_main = output_dim_type
hidden_dim_main = 64
output_dim_main = 1

In [269]:
learning_rate = 1e-2

In [270]:
#criterion = nn.MSELoss()
#model = Framework(seq_len, hidden_dim_type, output_dim_type, adj_matrix, hidden_dim_1, hidden_dim_2, output_dim_adj, k, input_dim_main, hidden_dim_main,output_dim_main)
model = Linear(8,32,1)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [271]:
def quantile_loss(y_true, y_pred, quantile):
    error = y_true - y_pred
    loss = torch.max(quantile * error, (quantile - 1) * error)
    return loss.mean()

In [326]:
model = Linear(8,32,1)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
model = model.double()
num_epochs = 30
quantile = 0.9
for epoch in range(num_epochs):
    running_loss = 0
    for i in range(X.shape[0]):
        inputs = X[i,:,:,:]
        inputs = inputs.permute(2,1,0)
        labels = y[i,:,:,:]
        labels = labels.permute(2,1,0)
        outputs = model(inputs)
        loss = quantile_loss(labels,outputs,quantile)

            # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

            # Update the running loss
        running_loss += loss.item()

        # Print the average loss for the epoch
    epoch_loss = running_loss / X.shape[0]
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")


    

Epoch 1/30, Loss: 0.18466472983149296
Epoch 2/30, Loss: 0.09025995660454827
Epoch 3/30, Loss: 0.08488789093030603
Epoch 4/30, Loss: 0.08301902384323276
Epoch 5/30, Loss: 0.0813967119836051
Epoch 6/30, Loss: 0.07986565988056392
Epoch 7/30, Loss: 0.07842189509477727
Epoch 8/30, Loss: 0.07705856925575234
Epoch 9/30, Loss: 0.07576786133897331
Epoch 10/30, Loss: 0.07453017846676477
Epoch 11/30, Loss: 0.07336567940921891
Epoch 12/30, Loss: 0.07225903314760518
Epoch 13/30, Loss: 0.07125749467454605
Epoch 14/30, Loss: 0.07048016603609816
Epoch 15/30, Loss: 0.07008490830613616
Epoch 16/30, Loss: 0.06998084875143741
Epoch 17/30, Loss: 0.06992199143292038
Epoch 18/30, Loss: 0.06988272291032736
Epoch 19/30, Loss: 0.06984810425354465
Epoch 20/30, Loss: 0.06981583465417554
Epoch 21/30, Loss: 0.06978576670941875
Epoch 22/30, Loss: 0.06975699144875555
Epoch 23/30, Loss: 0.06972914543707467
Epoch 24/30, Loss: 0.06970172416985024
Epoch 25/30, Loss: 0.06967486567016132
Epoch 26/30, Loss: 0.06964845767279

In [327]:
x = X[0,:,:,:]
Y = y[0,:,:,:]
x = x.permute(2,1,0)
Y = Y.permute(2,1,0)
outputs = model(x)
outputs.shape
print(outputs[0,0,0])

tensor(1.0001, dtype=torch.float64, grad_fn=<SelectBackward0>)


tensor(1., dtype=torch.float64)

In [286]:
model = Linear(8,32,1)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
model = model.double()
num_epochs = 30
quantile = 0.1
for epoch in range(num_epochs):
    running_loss = 0
    for i in range(X.shape[0]):
        inputs = X[i,:,:,:]
        inputs = inputs.permute(2,1,0)
        labels = y[i,:,:,:]
        labels = labels.permute(2,1,0)
        outputs = model(inputs)
        loss = quantile_loss(labels,outputs,quantile)

            # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

            # Update the running loss
        running_loss += loss.item()

        # Print the average loss for the epoch
    epoch_loss = running_loss / X.shape[0]
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")


    

Epoch 1/30, Loss: 0.04269981439246525
Epoch 2/30, Loss: 0.03892350016362153
Epoch 3/30, Loss: 0.03648743339206173
Epoch 4/30, Loss: 0.035305437111997906
Epoch 5/30, Loss: 0.03466622829106906
Epoch 6/30, Loss: 0.034305920620383
Epoch 7/30, Loss: 0.034089073234392274
Epoch 8/30, Loss: 0.033927833704458586
Epoch 9/30, Loss: 0.03378522977036841
Epoch 10/30, Loss: 0.03365537963917146
Epoch 11/30, Loss: 0.03353747728682481
Epoch 12/30, Loss: 0.03343538873209557
Epoch 13/30, Loss: 0.033339480709038936
Epoch 14/30, Loss: 0.03324774962992209
Epoch 15/30, Loss: 0.033162286464586105
Epoch 16/30, Loss: 0.03307993669673301
Epoch 17/30, Loss: 0.03300196616719869
Epoch 18/30, Loss: 0.032928831454653855
Epoch 19/30, Loss: 0.03286521458165528
Epoch 20/30, Loss: 0.03281056275442542
Epoch 21/30, Loss: 0.03275422962913563
Epoch 22/30, Loss: 0.03270204058628107
Epoch 23/30, Loss: 0.032652281283085595
Epoch 24/30, Loss: 0.03260717415071609
Epoch 25/30, Loss: 0.032563185654584804
Epoch 26/30, Loss: 0.0325242

In [302]:
x = X[0,:,:,:]
Y = y[0,:,:,:]
x = x.permute(2,1,0)
Y = Y.permute(2,1,0)
outputs = model(x)
print(outputs.shape)
print(outputs[8,0,0])

torch.Size([8, 113, 1])


IndexError: index 8 is out of bounds for dimension 0 with size 8

In [298]:
print(Y[7,0,0])

tensor(1., dtype=torch.float64)


In [250]:
y_pred = torch.from_numpy(np.array([[1,2,3],[4,5,6]]))
y_true = torch.from_numpy(np.array([[1,1,1],[1,1,1]]))

In [328]:
"""
Stochastic gradient MCMC implementation based on Diffusion Net
"""

import sys, copy
import numpy as np
import torch
import random
from torch.autograd import Variable

class Sampler:
    def __init__(self, net, criterion, momentum=0.0, lr=0.1, wdecay=0.0, T=0.05, total=50000, L2_weight = 0.0):
        self.net = net
        self.eta = lr
        self.momentum = momentum
        self.T = T
        self.wdecay = wdecay
        self.V = 0.1
        self.criterion = criterion
        self.total = total
        self.L2_weight = L2_weight

        print("Learning rate: ")
        print(self.eta)
        print("Noise std: ")
        print(self.scale)
        print("L2 penalty:")
        print(self.L2_weight)

    
    def backprop(self, x, y, batches_seen):
        self.net.zero_grad()
        """ convert mean loss to sum losses """
        output = self.net(x, y, batches_seen)
        loss = self.criterion(y, output) * self.total
        loss.backward()
        return loss 
    
    # SGD without momentum
    def step(self, x, y, batches_seen):
        loss = self.backprop(x, y, batches_seen)
        for i, param in enumerate(self.net.parameters()):
            proposal = torch.cuda.FloatTensor(param.data.size()).normal_().mul((self.eta*2.0)**0.5)
            proposal.add_(-0.5*self.scale, param.data)
            grads = param.grad.data
            if self.wdecay != 0:
                grads.add_(self.wdecay, param.data)
            # self.velocity[i].mul_(self.momentum).add_(-self.eta, grads).add_(proposal)
            # param.data.add_(self.velocity[i])
            param.data.add_(-self.eta, grads).add_(proposal)
        return loss.data.item()

In [ ]:
def _train(base_lr,
               steps, patience=50, epochs=100, lr_decay_ratio=0.1, log_every=1, save_model=1,
               test_every_n_epochs=10, epsilon=1e-8):
        # steps is used in learning rate - will see if need to use it?
        min_val_loss = float('inf')
        wait = 0
        # Initially setup some parameters
        sampler = None

      
        # this will fail if model is loaded with a changed batch_size
        num_batches = _data['train_loader'].num_batch
       

        batches_seen = num_batches * _epoch_num
        base_lr *= 0.1

        for epoch_num in range(_epoch_num, epochs):

            dcrnn_model = dcrnn_model.train()

            train_iterator = _data['train_loader'].get_iterator()
            losses = []



            for _, (x, y) in enumerate(train_iterator):
                x, y = _prepare_data(x, y)
                
                if batches_seen == 0: 
                    sampler = Sampler(dcrnn_model, _compute_loss, momentum=0.0, lr=base_lr, wdecay=0.0, T=0.01, total=23872, L2_weight = 0.3)

                loss = sampler.step(x, y, batches_seen)

                _logger.debug(loss / 23872.0)

                losses.append(loss / 23872.0)

                batches_seen += 1

                # gradient clipping - this does it in place
                torch.nn.utils.clip_grad_norm_(dcrnn_model.parameters(), max_grad_norm)

            self._logger.info("epoch complete")
            # lr_scheduler.step()
            """ Anneaing """
            if epoch_num > (0.04 * self.sn) and self.lr_anneal <= 1.:
                sampler.eta *= self.lr_anneal


        

            # self._writer.add_scalar('training loss',
            #                         np.mean(losses),
            #                         batches_seen)

            

In [330]:
import numpy as np
from sklearn.cluster import SpectralClustering

# Example adjacency matrix representing the graph
adjacency_matrix = np.array([[0, 1, 1, 0, 0],
                            [1, 0, 1, 0, 0],
                            [1, 1, 0, 1, 0],
                            [0, 0, 1, 0, 1],
                            [0, 0, 0, 1, 0]])

# Number of desired partitions
n_partitions = 3

# Perform spectral clustering
sc = SpectralClustering(n_clusters=n_partitions, affinity='precomputed', random_state=42)
labels = sc.fit_predict(adjacency_matrix)

# Print the assigned labels for each node
print("Node Labels:")
for i, label in enumerate(labels):
    print(f"Node {i}: Label {label}")


Node Labels:
Node 0: Label 1
Node 1: Label 1
Node 2: Label 0
Node 3: Label 0
Node 4: Label 2


In [18]:
import numpy as np
from sklearn.cluster import SpectralClustering

# Example adjacency matrix representing the graph
adjacency_matrix = np.array([[1, 0.5, 0, 0, 0],
                            [1, 1, 1, 0, 0],
                            [0, 1, 1, 1, 0],
                            [0, 0, 1, 1, 1],
                            [0, 0, 0, 1, 1]])
adjacency_matrix = np.load('/Users/pipipu/Desktop/HAGEN/HAGEN-code/crime-data/sensor_graph/adj_mx_la.pkl',allow_pickle = True)[2]
# Number of desired partitions
n_partitions = 2
#data = np.array([[1,2,3,4,5],[1,2,3,4,5]])
# Perform spectral clustering
sc = SpectralClustering(n_clusters=n_partitions, affinity='precomputed', random_state=42)
labels = sc.fit_predict(adjacency_matrix)


# Create sub-adjacency matrices
sub_adjacency_matrices = []
data_matrices = []

for label in np.unique(labels):
    
    sub_adjacency_matrix = adjacency_matrix[labels == label][:, labels == label]
    sub_adjacency_matrices.append(sub_adjacency_matrix)
    #data_matrix = data[:,labels == label]
    #data_matrices.append(data_matrix)
    
# Print the sub-adjacency matrices
print("Sub-Adjacency Matrices:")
for i, sub_adjacency_matrix in enumerate(sub_adjacency_matrices):
    print(f"Sub-Adjacency Matrix {i+1}:\n{sub_adjacency_matrix}")

#print(data_matrices)


Sub-Adjacency Matrices:
Sub-Adjacency Matrix 1:
[[1.         1.         0.374108   ... 0.21471389 0.9955341  0.7400604 ]
 [1.         1.         0.5280353  ... 0.24442069 0.8320746  0.9150456 ]
 [0.374108   0.5280353  1.         ... 0.8042993  0.13124737 0.8469326 ]
 ...
 [0.21471389 0.24442069 0.8042993  ... 1.         0.         0.4348732 ]
 [0.9955341  0.8320746  0.13124737 ... 0.         1.         0.3648845 ]
 [0.7400604  0.9150456  0.8469326  ... 0.4348732  0.3648845  1.        ]]
Sub-Adjacency Matrix 2:
[[1.         0.9928625  0.90650445 ... 0.3050112  0.57730895 0.33258843]
 [0.9928625  1.         0.4299091  ... 0.         0.16850382 0.4319986 ]
 [0.90650445 0.4299091  1.         ... 0.76377624 1.         0.        ]
 ...
 [0.3050112  0.         0.76377624 ... 1.         1.         0.        ]
 [0.57730895 0.16850382 1.         ... 1.         1.         0.        ]
 [0.33258843 0.4319986  0.         ... 0.         0.         1.        ]]


In [36]:
def matrix_partition(data,adjacency_matrix,n_partitions):
    sc = SpectralClustering(n_clusters=n_partitions, affinity='precomputed', random_state=42)
    labels = sc.fit_predict(adjacency_matrix)
    sub_adjacency_matrices = []
    data_matrices = []
    for label in np.unique(labels):
        sub_adjacency_matrix = adjacency_matrix[labels == label][:, labels == label]
        print(adjacency_matrix[labels == label])
        sub_adjacency_matrices.append(sub_adjacency_matrix)
        data_matrix = data[:,labels == label]
        data_matrices.append(data_matrix)
    
    seq_len,num_nodes = data.shape
    
    return sub_adjacency_matrices, data_matrices, labels

In [65]:
adjacency_matrix = np.array([[1, 0.5, 0, 0, 0],
                            [1, 1, 1, 0, 0],
                            [0, 1, 1, 1, 0],
                            [0, 0, 1, 1, 1],
                            [0, 0, 0, 1, 1]])

data = np.array([[1,2,3,4,5],[1,2,3,4,5]])

n_partitions = 3

sub_adjacency_matrices, data_matrices, labels = matrix_partition(data,adjacency_matrix,n_partitions)

[[0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1.]]
[[1.  0.5 0.  0.  0. ]]
[[1. 1. 1. 0. 0.]
 [0. 1. 1. 1. 0.]]


/Users/pipipu/miniconda3/envs/DLML/lib/python3.9/site-packages/sklearn/manifold/_spectral_embedding.py:248: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)


In [66]:
def data_reconstruct(labels,data_matrices,seq_len,num_nodes):

    unique_labels = np.unique(labels)
    k = len(unique_labels)

    n = seq_len
    m = num_nodes

    recovered_matrix = np.zeros((n, m))
    for i, small_matrix in enumerate(data_matrices):
        indices = np.where(labels == unique_labels[i])[0]
        recovered_matrix[:,indices] = small_matrix
        
    return recovered_matrix


In [67]:
data_reconstruct(labels, data_matrices, 2, 5)

array([[1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.]])

In [41]:
print(data_matrices)

[]


In [49]:
import numpy as np

# Example matrix and labels
matrix = np.array([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9],
                   [10, 11, 12]])
labels = np.array([0, 1, 2, 1])

# Determine the unique labels and the number of rows and columns in the matrix
unique_labels = np.unique(labels)
k = len(unique_labels)
n, m = matrix.shape

# Separate the big matrix into smaller matrices based on labels
small_matrices = []
for label in unique_labels:
    small_matrix = matrix[labels == label]
    small_matrices.append(small_matrix)

print(small_matrices)
# Print the smaller matrices
print("Smaller Matrices:")
for i, small_matrix in enumerate(small_matrices):
    print(f"Small Matrix {i+1}:\n{small_matrix}")

# Recover the original matrix
recovered_matrix = np.zeros((n, m))
for i, small_matrix in enumerate(small_matrices):
    
    indices = np.where(labels == unique_labels[i])[0]
    print(indices)
    recovered_matrix[indices] = small_matrix

print("\nRecovered Matrix:")
print(recovered_matrix)


[array([[1, 2, 3]]), array([[ 4,  5,  6],
       [10, 11, 12]]), array([[7, 8, 9]])]
Smaller Matrices:
Small Matrix 1:
[[1 2 3]]
Small Matrix 2:
[[ 4  5  6]
 [10 11 12]]
Small Matrix 3:
[[7 8 9]]
[0]
[1 3]
[2]

Recovered Matrix:
[[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8.  9.]
 [10. 11. 12.]]


In [ ]:
def matrix_partition()